In [5]:
     
import requests
from bs4 import BeautifulSoup
import selenium
import pandas
from selenium import webdriver
from datetime import datetime, date, time
import time
 

# url = "https://www.glassdoor.ie/Reviews/Aptiv-Reviews-E1891089.htm?filter.iso3Language=eng"
# driver.get(url)

noofpages = range(1, 53,1)

for page in noofpages:
    options = webdriver.ChromeOptions()
    options.add_argument("--ignore-certificate-errors")
    options.add_argument("--incognito")
    #options.add_argument('--headless')
    options.add_experimental_option('excludeSwitches', ['enable-logging'])
    driver = webdriver.Chrome(options=options)
    driver.implicitly_wait(100)
 
     
    url = "https://www.glassdoor.ie/Reviews/Aptiv-Reviews-E1891089_P{}.htm?filter.iso3Language=eng"
    url = url.format(page)
    
    driver.get(url)
    html_soup = BeautifulSoup(driver.page_source, "html.parser")
    containers = html_soup.find_all("div", class_="gdReview")
    
    userEmail = "hassan.tariq@aptiv.com"
    userPassword = "REKucuV4"
    
    driver.find_element_by_xpath("/html/body/header/nav[1]/div/div/div/div[1]/button").click()
    time.sleep(5)
    driver.find_element_by_xpath("/html/body/div[11]/div/div/div[2]/div[2]/div[2]/div/div/div/div[3]/form/div[1]/div/div/input").send_keys(userEmail)
    driver.find_element_by_xpath("/html/body/div[11]/div/div/div[2]/div[2]/div[2]/div/div/div/div[3]/form/div[2]/div/div/input").send_keys(userPassword)
    driver.find_element_by_xpath("/html/body/div[11]/div/div/div[2]/div[2]/div[2]/div/div/div/div[3]/form/div[3]/div[1]/button").click()
    time.sleep(5)
    # read = driver.find_element_by_class_name("v2__EIReviewDetailsV2__continueReading v2__EIReviewDetailsV2__clickable v2__EIReviewDetailsV2__newUiCta mb")
    # read.click()
    
    for container in containers:
    
        review_title = container.find('a', class_="reviewLink")
        review_title = review_title.text
        print('Title: ', review_title)
    
        rating = container.find('span', class_='ratingNumber mr-xsm')
        rating = rating.text
        print('Rating: ', rating)
    
        employee_Status = container.find('span', class_='pt-xsm pt-md-0 css-1qxtz39 eg4psks0')
        employee_Status = employee_Status.text
        print('Designation: ', employee_Status)
    
        rev_text = container.find('div', class_="v2__EIReviewDetailsV2__fullWidth")
        pros = rev_text.text
        pros = pros.strip('Pros')
        cons = rev_text.next_sibling.text
        cons = cons.strip('Cons')
        #advise = rev_text.next_sibling.next_sibling.text
        #advise = advise.strip('Advice to Management')
        print('pros: ', pros)
        print('cons: ', cons)
        #print('advise: ', advise)
    
    
        date_posted = container.find('span', class_='authorJobTitle middle common__EiReviewDetailsStyle__newGrey')
        date_posted = date_posted.text
        print('date & position: ', date_posted)
    
        # author_info = container.find('span', class_='authorLocation')
        # author_info = author_info.text
        # print('Author Info: ', author_info)
    
        print()

Title:  Great Culture and Company That Cares
Rating:  4.0
Designation:  Current Employee
pros:  Not much politics and very few ego's. Everyone is willing to help which is refreshing. Not very tough recruiting A Player Talent as Aptiv is a place where most want to work.
cons:  Need to focus on career growth and promoting within. Not much movement with internal candidates which turns into employees leaving.
date & position:  13 Jan 2021 - Senior Corporate Recruiter

Title:  Shaping the Future of Mobility!
Rating:  5.0
Designation:  Current Employee, more than 1 year
pros:  Global Work Environment, Cutting Edge Technology Sector, Interesting Challenge
cons:  None to mention at this time.
date & position:  14 Apr 2021 - Program Manager

Title:  Lots of potential- getting there
Rating:  4.0
Designation:  Current Employee, more than 1 year
pros:  A very fast paced environment - you definitely won't be bored and you learn lots in a short space of time
A very global company- exciting to work w

KeyboardInterrupt: 

In [23]:
Data = pd.DataFrame(
    {
        "Title": review_title,
        "Rating": rating,
        "Designation": employee_Status,
        "Pros": pros,
        "Cons": cons,
        "Author Info": author_info 
    }
    )   
Data.to_csv("{}.csv".format(review_title))
driver.quit()